In [1]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.nn import functional as F
#from torch.utils.data import Dataset
from datasets import Dataset

# 读取并预处理数据集
df = pd.read_csv('validation_labels.csv')
# 将所有列中的 NaN 替换为 0
df = df.fillna(0)
df

2025-03-09 16:10:57.152423: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 16:10:57.164342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 16:10:57.178655: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 16:10:57.182986: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 16:10:57.194272: I tensorflow/core/platform/cpu_feature_guar

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,z_37,x_38,y_38,z_38,x_39,y_39,z_39,x_40,y_40,z_40
0,R1107_1,G,1,-5.499000,8.520000,8.605000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
1,R1107_2,G,2,-5.826000,10.453000,14.010000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2,R1107_3,G,3,-5.849000,14.768000,17.584999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
3,R1107_4,G,4,-5.784000,19.985001,18.666000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
4,R1107_5,G,5,-5.755000,25.533001,17.132999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,R1190_114,U,114,87.870003,105.432999,115.183998,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2511,R1190_115,U,115,92.911003,105.394997,113.741997,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2512,R1190_116,U,116,99.012001,105.749001,113.073997,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2513,R1190_117,U,117,103.861000,103.453003,114.589996,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18


In [2]:
df['molecule_id'] = df['ID'].apply(lambda x: x.split('_')[0])
df['residue_id'] = df['resid']
df

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,y_38,z_38,x_39,y_39,z_39,x_40,y_40,z_40,molecule_id,residue_id
0,R1107_1,G,1,-5.499000,8.520000,8.605000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,1
1,R1107_2,G,2,-5.826000,10.453000,14.010000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,2
2,R1107_3,G,3,-5.849000,14.768000,17.584999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,3
3,R1107_4,G,4,-5.784000,19.985001,18.666000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,4
4,R1107_5,G,5,-5.755000,25.533001,17.132999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,R1190_114,U,114,87.870003,105.432999,115.183998,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1190,114
2511,R1190_115,U,115,92.911003,105.394997,113.741997,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1190,115
2512,R1190_116,U,116,99.012001,105.749001,113.073997,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1190,116
2513,R1190_117,U,117,103.861000,103.453003,114.589996,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1190,117


In [3]:
groups = df.groupby('molecule_id')
data = []
for molecule_id, group in groups:
    group = group.sort_values('residue_id')
    sequence = ''.join(group['resname'].values)
    labels = group[['x_1', 'y_1', 'z_1']].values.tolist()
    pos_data = {
        "seq":sequence,
        "pos_list":labels
    }
    data.append(pos_data)

data[0]

{'seq': 'GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU',
 'pos_list': [[-5.499000072479248, 8.520000457763672, 8.604999542236328],
  [-5.826000213623047, 10.45300006866455, 14.010000228881836],
  [-5.848999977111816, 14.767999649047852, 17.584999084472656],
  [-5.783999919891357, 19.98500061035156, 18.666000366210938],
  [-5.755000114440918, 25.53300094604492, 17.132999420166016],
  [-6.227000236511231, 30.093000411987305, 13.96500015258789],
  [-9.015999794006348, 37.02999877929688, 11.305999755859377],
  [-9.026000022888184, 31.554000854492188, 8.725000381469727],
  [-13.911999702453612, 30.90800094604492, 8.347000122070312],
  [-22.27300071716309, 33.250999450683594, 7.105000019073486],
  [-25.75200080871582, 28.85400009155273, 8.54800033569336],
  [-28.566999435424805, 25.027000427246097, 6.709000110626221],
  [-30.613000869750977, 22.207000732421875, 2.5999999046325684],
  [-30.474000930786133, 20.333999633789062, -2.325999975204468],
  [-27.76700019836425,

In [4]:
import json
filename = "rna_pos_all_val.jsonl"
rna_file = open(filename,"w")
for item in data:
    rna_file.write(json.dumps(item) + "\n")

rna_file.close()